# COURSERA CAPSTONE - Best place for a restaurant in Paris (France)

### This Jupyter Notebook is used for the capstone project Assignment week 4 and 5.
    -Start date : 13/08/2021
    -End date : 30/08/2021

### Sources
   - [Borough and neighborhood of Paris](https://fr.wikipedia.org/wiki/Liste_des_quartiers_administratifs_de_Paris)
    

### Import Libraries

In [1]:
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import re

import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

import requests # library to handle requests
from bs4 import BeautifulSoup # library to handle HTML page 

import geocoder # import geocoder to handle geographical coordinates
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import folium # map rendering library

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

##  1) Scraping Data from Wikipedia page and create Dataframe

#### Get Data from "Paris" (Wikipedia page) with "BeautifulSoup

In [2]:
url = "https://fr.wikipedia.org/wiki/Liste_des_quartiers_administratifs_de_Paris"

In [3]:
html_data = requests.get(url).text 
soup = BeautifulSoup(html_data,"html5lib")

In [4]:
table_contents=[]
table = soup.find('table')

#### Dataframe creation

In [5]:
dfs = pd.read_html(str(table))
df = dfs[0]
df

,"Arrondissement[1],[n 1]",Quartiers,Quartiers.1,Population en1999 (hab.)[2],Superficie(ha)[2],Densitéhab/km2,Plan
0,1er arrondissementdit « du Louvre »,1er,Saint-Germain-l'Auxerrois,1 672,869,1 924,NaN
1,1er arrondissementdit « du Louvre »,2e,Halles,8 984,412,21 806,NaN
2,1er arrondissementdit « du Louvre »,3e,Palais-Royal,3 195,274,11 661,NaN
3,1er arrondissementdit « du Louvre »,4e,Place-Vendôme,3 044,269,11 316,NaN
4,2e arrondissementdit « de la Bourse »,5e,Gaillon,1 345,188,7 154,NaN
...,...,...,...,...,...,...,...
75,19e arrondissementdit « des Buttes-Chaumont »,76e,Combat,38 988,1295,30 107,NaN
76,20e arrondissementdit « de Ménilmontant »,77e,Belleville,35 773,807,44 328,NaN
77,20e arrondissementdit « de Ménilmontant »,78e,Saint-Fargeau,42 087,1487,28 303,NaN
78,20e arrondissementdit « de Ménilmontant »,79e,Père-Lachaise,42 332,1599,26 474,NaN


#### a few borough replacements and some neighborhood adjustments. 

In [6]:
#Cleaning colum names and drop inuse columns
df.rename(columns={'Arrondissement[1],[n 1]' : 'Borough', 
                   'Quartiers.1' : 'Neighborhood',
                   'Population en1999 (hab.)[2]' : 'Inhabitant',
                   'Superficie(ha)[2]' : 'Area'}, inplace = True)
df.drop(['Quartiers', 'Densitéhab/km2', 'Plan'], axis =1, inplace = True)
df = df.replace({'Borough': r'\s.(.*)'}, {'Borough': ''}, regex=True)
df

,Borough,Neighborhood,Inhabitant,Area
0,1er,Saint-Germain-l'Auxerrois,1 672,869
1,1er,Halles,8 984,412
2,1er,Palais-Royal,3 195,274
3,1er,Place-Vendôme,3 044,269
4,2e,Gaillon,1 345,188
...,...,...,...,...
75,19e,Combat,38 988,1295
76,20e,Belleville,35 773,807
77,20e,Saint-Fargeau,42 087,1487
78,20e,Père-Lachaise,42 332,1599


## 2) Localisation of each neighborhood 

#### Creation of the Lat. & Long .columns

In [7]:
df['Latitude'] = np.nan
df['Longitude'] = np.nan
Neighborhood = df['Neighborhood']

#### Loop to call geocoder with postal code in parameter  and assigning  the Latitude and Longitude in the DF.

In [8]:
for i, pc in enumerate(Neighborhood):
    g = geocoder.arcgis('{}, Paris, France'.format(pc))
    df.loc[i, 'Latitude'] = g.latlng[0]
    df.loc[i, 'Longitude'] = g.latlng[1]

In [9]:
df.head()

,Borough,Neighborhood,Inhabitant,Area,Latitude,Longitude
0,1er,Saint-Germain-l'Auxerrois,1 672,869,48.85971,2.34024
1,1er,Halles,8 984,412,48.86319,2.34201
2,1er,Palais-Royal,3 195,274,48.86350,2.33876
3,1er,Place-Vendôme,3 044,269,48.86778,2.33011
4,2e,Gaillon,1 345,188,48.86902,2.33452


## 3) Exploring and clustering the Neighborhood

#### Foursquare Credentials and Version

In [10]:
CLIENT_ID = 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX' #  Foursquare ID
CLIENT_SECRET = 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX' #  Foursquare Secret
ACCESS_TOKEN = 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX' #  FourSquare Access Token
VERSION = '20180604'
LIMIT = 50

#### Create a map of Paris with neighborhoods superimposed on top.

In [11]:
# Map initialization on Paris localisation
address = 'Paris, France'
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

In [12]:
# create map of Paris using latitude and longitude values
map_paris = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, bor, neigh in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{} {}'.format(bor, neigh)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker([lat, lng], radius=5, popup=label, color='blue',fill=True, 
                        fill_color='#3186cc', fill_opacity=0.7, parse_html=False).add_to(map_paris)  
    
map_paris

#### Localisation of all venues in Paris using Foursquare API

Use a function to get venues for all the neighborhoods in Paris

In [13]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    """
    Function for getting all venues given in parameter "names"
    Return a DF with venue name, venue latitude & longitude, and its category (for each location name passed in parameters)
    """
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):       
        # create URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={}, \
              {}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT)
            
        # GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(name, lat, lng, v['venue']['name'], v['venue']['location']['lat'], \
                             v['venue']['location']['lng'],  v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 'Neighborhood Latitude', 'Neighborhood Longitude', 'Venue', \
                             'Venue Latitude', 'Venue Longitude', 'Venue Category']
    
    return(nearby_venues)

In [14]:
paris_venues = getNearbyVenues(names=df['Neighborhood'], latitudes=df['Latitude'],longitudes=df['Longitude'])

Check how many venues were returned for each neighborhood

In [15]:
paris_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Amérique,35,35,35,35,35,35
Archives,50,50,50,50,50,50
Arsenal,50,50,50,50,50,50
Arts-et-Métiers,15,15,15,15,15,15
Auteuil,45,45,45,45,45,45
...,...,...,...,...,...,...
Val-de-Grâce,46,46,46,46,46,46
Villette,50,50,50,50,50,50
Vivienne,50,50,50,50,50,50


## 3. Analyze Each Neighborhood

Encoding of "venue category" with a numerical variable

In [16]:
# one hot encoding
paris_onehot = pd.get_dummies(paris_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
paris_onehot['Neighborhood'] = paris_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [paris_onehot.columns[-1]] + list(paris_onehot.columns[:-1])
paris_onehot = paris_onehot[fixed_columns]

Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [17]:
paris_grouped = paris_onehot.groupby('Neighborhood').mean().reset_index()
paris_grouped

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Alsatian Restaurant,American Restaurant,Arcade,Argentinian Restaurant,Art Gallery,...,Trattoria/Osteria,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio,Zoo Exhibit
0,Amérique,0.0,0.0,0.0,0.028571,0.0,0.00,0.00,0.00,0.00,...,0.0,0.000000,0.0,0.00,0.00,0.000000,0.028571,0.0,0.0,0.0
1,Archives,0.0,0.0,0.0,0.000000,0.0,0.00,0.00,0.02,0.04,...,0.0,0.000000,0.0,0.00,0.06,0.060000,0.000000,0.0,0.0,0.0
2,Arsenal,0.0,0.0,0.0,0.000000,0.0,0.00,0.00,0.00,0.00,...,0.0,0.000000,0.0,0.02,0.00,0.020000,0.020000,0.0,0.0,0.0
3,Arts-et-Métiers,0.0,0.0,0.0,0.000000,0.0,0.00,0.00,0.00,0.00,...,0.0,0.000000,0.0,0.00,0.00,0.000000,0.000000,0.0,0.0,0.0
4,Auteuil,0.0,0.0,0.0,0.000000,0.0,0.00,0.00,0.00,0.00,...,0.0,0.000000,0.0,0.00,0.00,0.000000,0.022222,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,Val-de-Grâce,0.0,0.0,0.0,0.000000,0.0,0.00,0.00,0.00,0.00,...,0.0,0.021739,0.0,0.00,0.00,0.021739,0.000000,0.0,0.0,0.0
76,Villette,0.0,0.0,0.0,0.000000,0.0,0.02,0.00,0.00,0.00,...,0.0,0.000000,0.0,0.00,0.00,0.000000,0.000000,0.0,0.0,0.0
77,Vivienne,0.0,0.0,0.0,0.000000,0.0,0.02,0.02,0.00,0.00,...,0.0,0.000000,0.0,0.00,0.00,0.060000,0.000000,0.0,0.0,0.0
78,École-Militaire,0.0,0.0,0.0,0.000000,0.0,0.00,0.00,0.00,0.00,...,0.0,0.000000,0.0,0.00,0.00,0.000000,0.000000,0.0,0.0,0.0


 Print each neighborhood along with the top 10 of the most common venues

In [18]:
num_top_venues = 8

for hood in paris_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = paris_grouped[paris_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Amérique----
                 venue  freq
0                  Bar  0.14
1    French Restaurant  0.09
2               Bistro  0.06
3           Restaurant  0.06
4  Japanese Restaurant  0.06
5               Bakery  0.06
6                 Café  0.03
7          Supermarket  0.03


----Archives----
                   venue  freq
0                 Bistro  0.06
1         Sandwich Place  0.06
2      French Restaurant  0.06
3               Wine Bar  0.06
4  Vietnamese Restaurant  0.06
5     Italian Restaurant  0.06
6                  Hotel  0.04
7              Bookstore  0.04


----Arsenal----
               venue  freq
0  French Restaurant  0.20
1              Hotel  0.10
2              Plaza  0.06
3   Pedestrian Plaza  0.04
4       Cocktail Bar  0.04
5        Coffee Shop  0.04
6   Tapas Restaurant  0.04
7           Tea Room  0.04


----Arts-et-Métiers----
                 venue  freq
0                 Park  0.13
1  Japanese Restaurant  0.07
2                Hotel  0.07
3              Theate

                venue  freq
0   French Restaurant  0.14
1              Bakery  0.11
2  Italian Restaurant  0.08
3              Garden  0.05
4         Supermarket  0.05
5               Trail  0.03
6      Sandwich Place  0.03
7            Tea Room  0.03


----Necker----
                venue  freq
0   French Restaurant  0.20
1               Hotel  0.15
2  Italian Restaurant  0.05
3        Dessert Shop  0.05
4   Korean Restaurant  0.05
5         Pastry Shop  0.02
6           Brasserie  0.02
7  Chinese Restaurant  0.02


----Notre-Dame----
                venue  freq
0   French Restaurant  0.16
1               Plaza  0.08
2      Ice Cream Shop  0.06
3         Coffee Shop  0.04
4           Bookstore  0.04
5              Bistro  0.04
6          Comic Shop  0.04
7  Mexican Restaurant  0.04


----Notre-Dame-des-Champs----
                venue  freq
0   French Restaurant  0.22
1               Hotel  0.12
2  Seafood Restaurant  0.04
3              Bakery  0.04
4    Asian Restaurant  0.04
5     

Transform that on a new DF

In [19]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [20]:
num_top_venues = 8

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = paris_grouped['Neighborhood']

for ind in np.arange(paris_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(paris_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head(60)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue
0,Amérique,Bar,French Restaurant,Bistro,Restaurant,Japanese Restaurant,Bakery,Café,Supermarket
1,Archives,Bistro,Sandwich Place,French Restaurant,Wine Bar,Vietnamese Restaurant,Italian Restaurant,Hotel,Bookstore
2,Arsenal,French Restaurant,Hotel,Plaza,Pedestrian Plaza,Cocktail Bar,Coffee Shop,Tapas Restaurant,Tea Room
3,Arts-et-Métiers,Park,Japanese Restaurant,Hotel,Theater,Pizza Place,BBQ Joint,Diner,Brasserie
4,Auteuil,French Restaurant,Italian Restaurant,Metro Station,Supermarket,Japanese Restaurant,Asian Restaurant,Plaza,Wine Shop
5,Batignolles,French Restaurant,Wine Bar,Italian Restaurant,Bar,Pastry Shop,Lebanese Restaurant,Restaurant,Japanese Restaurant
6,Bel-Air,French Restaurant,Hotel,Bakery,Gym / Fitness Center,Creperie,Pizza Place,Coffee Shop,Cheese Shop
7,Belleville,Bar,French Restaurant,Vegetarian / Vegan Restaurant,Pizza Place,Cocktail Bar,Café,Dim Sum Restaurant,Park
8,Bercy,Hotel,French Restaurant,Italian Restaurant,Museum,Wine Bar,Japanese Restaurant,Kitchen Supply Store,Pedestrian Plaza
9,Bonne-Nouvelle,French Restaurant,Hotel,Wine Bar,Cocktail Bar,Bar,Burger Joint,Beer Bar,Restaurant


## 4. Cluster Neighborhoods

Run k-means to cluster the neighborhood into 10 clusters.

In [21]:
# set number of clusters
kclusters = 8

paris_grouped_clustering = paris_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(paris_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([5, 2, 3, 2, 3, 3, 4, 2, 2, 1])

In [22]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

paris_merged = df

# merge neighborhoods_venues_sorted with df to add latitude/longitude for each neighborhood
paris_merged = paris_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

paris_merged.head()

,Borough,Neighborhood,Inhabitant,Area,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue
0,1er,Saint-Germain-l'Auxerrois,1 672,869,48.85971,2.34024,2,Plaza,Bar,Italian Restaurant,French Restaurant,Historic Site,Exhibit,Wine Bar,Clothing Store
1,1er,Halles,8 984,412,48.86319,2.34201,3,French Restaurant,Plaza,Café,Historic Site,Restaurant,Italian Restaurant,Bar,Hotel
2,1er,Palais-Royal,3 195,274,48.86350,2.33876,3,French Restaurant,Plaza,Café,Hotel,Historic Site,Bistro,Udon Restaurant,Art Museum
3,1er,Place-Vendôme,3 044,269,48.86778,2.33011,1,Hotel,Jewelry Store,French Restaurant,Cocktail Bar,Chocolate Shop,Boutique,Plaza,Hotel Bar
4,2e,Gaillon,1 345,188,48.86902,2.33452,1,Hotel,Japanese Restaurant,Korean Restaurant,Pastry Shop,Food & Drink Shop,Bookstore,Bakery,Italian Restaurant


### Visualize the resulting clusters

In [23]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(paris_merged['Latitude'], paris_merged['Longitude'], paris_merged['Neighborhood'],
                                  paris_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine each cluster and determine the neighborhood where restaurants are few and there are at least two cultural venue categories.

In [24]:
paris_merged.loc[paris_merged['Cluster Labels'] == 0, paris_merged.columns[[1] + list(range(5, paris_merged.shape[1]))]]

,Neighborhood,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue
32,Saint-Georges,2.37452,0,French Restaurant,Pool,Bar,Roof Deck,Southern / Soul Food Restaurant,Moroccan Restaurant,Bistro,Supermarket
79,Charonne,2.40419,0,French Restaurant,Supermarket,Bistro,Lounge,Tram Station,Gym,Gym / Fitness Center,Food & Drink Shop


In [25]:
paris_merged.loc[paris_merged['Cluster Labels'] == 1, paris_merged.columns[[1] + list(range(5, paris_merged.shape[1]))]]

,Neighborhood,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue
3,Place-Vendôme,2.330110,1,Hotel,Jewelry Store,French Restaurant,Cocktail Bar,Chocolate Shop,Boutique,Plaza,Hotel Bar
4,Gaillon,2.334520,1,Hotel,Japanese Restaurant,Korean Restaurant,Pastry Shop,Food & Drink Shop,Bookstore,Bakery,Italian Restaurant
5,Vivienne,2.341370,1,Hotel,French Restaurant,Wine Bar,Cocktail Bar,Pedestrian Plaza,Bistro,Pizza Place,Japanese Restaurant
7,Bonne-Nouvelle,2.347990,1,French Restaurant,Hotel,Wine Bar,Cocktail Bar,Bar,Burger Joint,Beer Bar,Restaurant
18,Val-de-Grâce,2.344430,1,French Restaurant,Hotel,Bistro,Greek Restaurant,Café,Beer Bar,Creperie,Church
20,Monnaie,2.343270,1,French Restaurant,Hotel,Bookstore,Chocolate Shop,Restaurant,Plaza,Creperie,Comic Shop
30,Madeleine,2.323800,1,Boutique,Hotel,Women's Store,Dessert Shop,French Restaurant,Tea Room,Pastry Shop,Sandwich Place
33,Chaussée-d'Antin,2.333060,1,Hotel,Clothing Store,French Restaurant,Coffee Shop,Salad Place,Department Store,Theater,Boutique
34,Faubourg-Montmartre,2.342041,1,Hotel,French Restaurant,Wine Bar,Japanese Restaurant,Bakery,Pedestrian Plaza,Sandwich Place,Corsican Restaurant
37,Porte-Saint-Denis,2.352720,1,Bar,Hotel,French Restaurant,Cocktail Bar,Beer Bar,Restaurant,Burger Joint,Indie Movie Theater


In [26]:
paris_merged.loc[paris_merged['Cluster Labels'] == 2, paris_merged.columns[[1] + list(range(5, paris_merged.shape[1]))]]

,Neighborhood,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue
0,Saint-Germain-l'Auxerrois,2.340240,2,Plaza,Bar,Italian Restaurant,French Restaurant,Historic Site,Exhibit,Wine Bar,Clothing Store
8,Arts-et-Métiers,2.334049,2,Park,Japanese Restaurant,Hotel,Theater,Pizza Place,BBQ Joint,Diner,Brasserie
9,Enfants-Rouges,2.361200,2,French Restaurant,Sandwich Place,Wine Bar,Art Gallery,Café,Bistro,Hotel,Bookstore
10,Archives,2.360860,2,Bistro,Sandwich Place,French Restaurant,Wine Bar,Vietnamese Restaurant,Italian Restaurant,Hotel,Bookstore
11,Sainte-Avoye,2.356058,2,French Restaurant,Restaurant,Art Gallery,Gourmet Shop,Garden,Pastry Shop,Coffee Shop,Ice Cream Shop
12,Saint-Merri,2.350530,2,Art Gallery,French Restaurant,Plaza,Restaurant,Ice Cream Shop,Art Museum,Park,Lebanese Restaurant
13,Saint-Gervais,2.353930,2,French Restaurant,Ice Cream Shop,Seafood Restaurant,Tea Room,Garden,Wine Bar,Plaza,Art Gallery
17,Jardin-des-Plantes,2.355670,2,French Restaurant,Garden,Greek Restaurant,Museum,Tea Room,Coffee Shop,Farmers Market,Botanical Garden
19,Sorbonne,2.344090,2,French Restaurant,Bookstore,Indie Movie Theater,Plaza,Japanese Restaurant,Hotel,Mexican Restaurant,Comic Shop
21,Odéon,2.338640,2,Seafood Restaurant,Bistro,French Restaurant,Fountain,Italian Restaurant,Ice Cream Shop,Chocolate Shop,Indie Movie Theater


In [27]:
paris_merged.loc[paris_merged['Cluster Labels'] == 3, paris_merged.columns[[1] + list(range(5, paris_merged.shape[1]))]]

,Neighborhood,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue
1,Halles,2.342010,3,French Restaurant,Plaza,Café,Historic Site,Restaurant,Italian Restaurant,Bar,Hotel
2,Palais-Royal,2.338760,3,French Restaurant,Plaza,Café,Hotel,Historic Site,Bistro,Udon Restaurant,Art Museum
14,Arsenal,2.366080,3,French Restaurant,Hotel,Plaza,Pedestrian Plaza,Cocktail Bar,Coffee Shop,Tapas Restaurant,Tea Room
15,Notre-Dame,2.349950,3,French Restaurant,Plaza,Ice Cream Shop,Coffee Shop,Bookstore,Bistro,Comic Shop,Mexican Restaurant
16,Saint-Victor,2.352970,3,French Restaurant,Bakery,Coffee Shop,Museum,Hotel,Café,Pub,Italian Restaurant
24,Saint-Thomas-d'Aquin,2.326780,3,French Restaurant,Hotel,Bakery,Italian Restaurant,Cheese Shop,Seafood Restaurant,Historic Site,Art Museum
25,Invalides,2.314880,3,French Restaurant,Garden,Plaza,Beer Garden,Bistro,Historic Site,Art Gallery,Hotel
27,Gros-Caillou,2.299010,3,French Restaurant,Italian Restaurant,Bistro,Hotel,Garden,Japanese Restaurant,Bakery,Burger Joint
28,Champs-Élysées,2.309930,3,French Restaurant,Art Gallery,Hotel,Plaza,Boutique,Cocktail Bar,Cosmetics Shop,Tailor Shop
35,Rochechouart,2.344579,3,French Restaurant,Pizza Place,Plaza,Italian Restaurant,Bar,Bakery,Coffee Shop,Concert Hall


In [28]:
paris_merged.loc[paris_merged['Cluster Labels'] == 4, paris_merged.columns[[1] + list(range(5, paris_merged.shape[1]))]]

,Neighborhood,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue
6,Mail,2.322300,4,Hotel,Japanese Restaurant,French Restaurant,Creperie,Pub,Bistro,Pizza Place,Comedy Club
22,Notre-Dame-des-Champs,2.334770,4,French Restaurant,Hotel,Seafood Restaurant,Bakery,Asian Restaurant,Steakhouse,Café,Gastropub
26,École-Militaire,2.303030,4,French Restaurant,Hotel,Coffee Shop,Creperie,Café,Plaza,Grocery Store,Ice Cream Shop
29,Faubourg-du-Roule,2.299606,4,Hotel,French Restaurant,Italian Restaurant,Tea Room,Seafood Restaurant,Sushi Restaurant,Clothing Store,Coffee Shop
31,Europe,2.325990,4,Hotel,French Restaurant,Wine Bar,Italian Restaurant,Pub,Bistro,Bookstore,Gym / Fitness Center
44,Bel-Air,2.296850,4,French Restaurant,Hotel,Bakery,Gym / Fitness Center,Creperie,Pizza Place,Coffee Shop,Cheese Shop
48,Salpêtrière,2.358300,4,Hotel,French Restaurant,Chinese Restaurant,Sushi Restaurant,Thai Restaurant,Japanese Restaurant,Italian Restaurant,Bar
52,Montparnasse,2.321290,4,French Restaurant,Japanese Restaurant,Hotel,Creperie,Pizza Place,Comedy Club,Italian Restaurant,Pub
54,Petit-Montrouge,2.325200,4,French Restaurant,Hotel,Italian Restaurant,Bakery,Bistro,Sushi Restaurant,Supermarket,Sandwich Place
57,Necker,2.313760,4,French Restaurant,Hotel,Italian Restaurant,Dessert Shop,Korean Restaurant,Pastry Shop,Brasserie,Chinese Restaurant


In [29]:
paris_merged.loc[paris_merged['Cluster Labels'] == 5, paris_merged.columns[[1] + list(range(5, paris_merged.shape[1]))]]

,Neighborhood,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue
70,Goutte-d'Or,2.353950,5,Bar,Indian Restaurant,Grocery Store,Bookstore,Supermarket,Theater,Pizza Place,Plaza
71,Chapelle,2.357046,5,Indian Restaurant,Grocery Store,Hotel,Supermarket,French Restaurant,Theater,Bakery,Bistro
74,Amérique,2.393520,5,Bar,French Restaurant,Bistro,Restaurant,Japanese Restaurant,Bakery,Café,Supermarket
77,Saint-Fargeau,2.403992,5,Bakery,Supermarket,French Restaurant,Plaza,Arts & Entertainment,Bistro,Chinese Restaurant,Electronics Store


In [30]:
paris_merged.loc[paris_merged['Cluster Labels'] == 6, paris_merged.columns[[1] + list(range(5, paris_merged.shape[1]))]]

,Neighborhood,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue
75,Combat,1.52147,6,Gym / Fitness Center,Shoe Store,Fast Food Restaurant,Health & Beauty Service,Furniture / Home Store,Chinese Restaurant,Park,Performing Arts Venue


In [31]:
paris_merged.loc[paris_merged['Cluster Labels'] == 7, paris_merged.columns[[1] + list(range(5, paris_merged.shape[1]))]]

,Neighborhood,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue
50,Maison-Blanche,2.35865,7,Vietnamese Restaurant,Asian Restaurant,Thai Restaurant,Chinese Restaurant,Cantonese Restaurant,Pastry Shop,Cambodian Restaurant,Gourmet Shop
